In [ ]:
import os
import numpy as np
import librosa

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical


In [4]:
def extract_mfcc(file_path, n_mfcc=40):
    audio, sample_rate = librosa.load(file_path, sr=None)
    mfcc = librosa.feature.mfcc(
        y=audio,
        sr=sample_rate,
        n_mfcc=n_mfcc
    )
    mfcc_mean = np.mean(mfcc.T, axis=0)
    return mfcc_mean


In [5]:
dataset_path = "Baby Cry Dataset"

X = []
y = []

for emotion in os.listdir(dataset_path):
    emotion_path = os.path.join(dataset_path, emotion)

    if not os.path.isdir(emotion_path):
        continue

    for file in os.listdir(emotion_path):
        if file.endswith(".wav"):
            file_path = os.path.join(emotion_path, file)
            mfcc_features = extract_mfcc(file_path)
            X.append(mfcc_features)
            y.append(emotion)


In [6]:
X = np.array(X)
y = np.array(y)


In [7]:
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

y_categorical = to_categorical(y_encoded)


In [8]:
X = X.reshape(X.shape[0], X.shape[1], 1)


In [ ]:
print(X.shape)


In [11]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y_categorical,
    test_size=0.2,
    random_state=42
)


In [ ]:
model = Sequential()

model.add(Conv1D(64, kernel_size=3, activation='relu', input_shape=(X.shape[1], 1)))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(128, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))

model.add(Dense(y_categorical.shape[1], activation='softmax'))


In [13]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


In [ ]:
history = model.fit(
    X_train,
    y_train,
    epochs=100,
    batch_size=32,
    validation_data=(X_test, y_test)
)


In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Accuracy:", accuracy)


In [ ]:
model.save("emotion_cnn_model_baby.h5")


In [17]:
import pickle

with open("label_encoder_baby.pkl", "wb") as f:
    pickle.dump(label_encoder, f)


In [ ]:
model.summary()